### Import libraries

In [51]:
import pandas as pd
import numpy as np

import talib as ta

import warnings
warnings.filterwarnings("ignore")

from datetime import datetime
import time

### Download data

In [ ]:
# import yfinance as yf

# tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA']

# start_date = "2020-01-01"
# end_date = datetime.now().strftime("%Y-%m-%d")

# # Download historical data from yf API
# data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')

# # download data to csv using ticker, exchange, date range and "raw" indicator in the filename
# data.to_csv(f"data_{'_'.join(tickers)}_{start_date}_{end_date}_raw.csv")

[*********************100%***********************]  7 of 7 completed


In [53]:
# Load the CSV with MultiIndex columns (Tickers, OHLCV)
df = pd.read_csv("../00_data/data_AAPL_MSFT_GOOGL_AMZN_TSLA_META_NVDA_2020-01-01_2025-10-26_raw.csv", header=[0,1], index_col=0)

# Drop any rows that are completely NaN (e.g. 'Date' row)
df = df.dropna(how='all')

# Convert all values to float
df = df.astype(float)

# Show the result
df.head()

Ticker            MSFT                                                  \
Price             Open        High         Low       Close      Volume   
Date                                                                     
2020-01-02  151.040856  152.895807  150.612792  152.791168  22622100.0   
2020-01-03  150.603275  152.153817  150.355939  150.888641  21116200.0   
2020-01-06  149.423644  151.345190  148.881420  151.278595  20813700.0   
2020-01-07  151.554487  151.887418  149.651970  149.899292  21634100.0   
2020-01-08  151.183463  152.962326  150.251234  152.286926  27746500.0   

Ticker           AMZN                                               ...  \
Price            Open       High        Low      Close      Volume  ...   
Date                                                                ...   
2020-01-02  93.750000  94.900497  93.207497  94.900497  80580000.0  ...   
2020-01-03  93.224998  94.309998  93.224998  93.748497  75288000.0  ...   
2020-01-06  93.000000  95.184502  93.000000  95.143997  81236000.0  ...   
2020-01-07  95.224998  95.694504  94.601997  95.343002  80898000.0  ...   
2020-01-08  94.902000  95.550003  94.321999  94.598503  70160000.0  ...   

Ticker            META                                                  \
Price             Open        High         Low       Close      Volume   
Date                                                                     
2020-01-02  205.483256  208.504623  205.006201  208.494690  12077100.0   
2020-01-03  205.940433  209.110876  205.682017  207.391479  11188400.0   
2020-01-06  205.433564  211.476314  205.254674  211.297424  17058900.0   
2020-01-07  211.516088  213.265300  210.452637  211.754608  14912400.0   
2020-01-08  211.694968  214.915122  211.307358  213.901367  13475000.0   

Ticker           AAPL                                                
Price            Open       High        Low      Close       Volume  
Date                                                                 
2020-01-02  71.545890  72.598892  71.292304  72.538513  135480400.0  
2020-01-03  71.765667  72.594055  71.608685  71.833290  146322800.0  
2020-01-06  70.954203  72.444336  70.703027  72.405693  118387200.0  
2020-01-07  72.415337  72.671341  71.845369  72.065147  108872000.0  
2020-01-08  71.768101  73.526318  71.768101  73.224426  132079200.0  

[5 rows x 35 columns]

### Random ticker selection

In [68]:
# randomly shuffle the level 0 of the columns (the tickers), to select one random tickers for strategy and becktesting
import random

random_ticker = random.choice(df.columns.levels[0])

print(f"Randomly selected ticker for strategy and backtesting: {random_ticker}")

Randomly selected ticker for strategy and backtesting: META


In [39]:
# calculate VWAP

In [40]:
# calculate ATR for stop loss and take profit

In [41]:
# list of indicators

In [42]:
# list of candlestick patterns

In [43]:
# select random indicator or candlestick pattern

In [44]:
# backtest VWAP strategy with random indicator

In [45]:
# backtest out of sample data

In [46]:
# create performance report with the backtest results for all strategies after out of sample backtest

In [47]:
# optimize the parameters of the 3 best strategies

In [48]:
# 3 best strategies after optimization should be tested on 1 hour data

In [49]:
# Monte Carlo simulation or bootstrap to evaluate the robustness of the strategies

In [50]:
# Final performance report